In [119]:
# Load Data
import pandas as pd
import numpy as np

VERBOSE = False

rotowire_file = "/home/delano/Google Drive/fantasy/nhl/external-projections/rotowire_all.csv"
rotogrinder_file = "/home/delano/Google Drive/fantasy/nhl/external-projections/grinder_all.csv"
fantasy_file = "/home/delano/scratch/nhl-prediction-dump.csv"

rotowire_df = pd.read_csv(rotowire_file)
rotogrinder_df = pd.read_csv(rotogrinder_file)
fantasy_df = pd.read_csv(fantasy_file)

In [120]:
rotowire_pts_df = rotowire_df.rename(columns={rotowire_df.columns[1]: 'player'})
rotowire_pts_df['service'] = rotowire_pts_df.apply(lambda row: 'rotowire:' + row[0].split('.')[1], axis=1)
rotowire_pts_df['date'] = pd.to_datetime(rotowire_pts_df.apply(lambda row: row[0].split('.')[2], axis=1))

rotowire_pts_df = rotowire_pts_df.pivot_table(index=['date', 'player'], values='FPTS', columns='service')
if VERBOSE:
    display(rotowire_pts_df)

In [121]:
rotogrinder_pts_df = rotogrinder_df
rotogrinder_pts_df['service'] = rotogrinder_pts_df.apply(lambda row: 'rotogrinder:' + row.file.split('.')[1], axis=1)
rotogrinder_pts_df['date'] = pd.to_datetime(rotogrinder_pts_df.apply(lambda row: row.file.split('.')[2], axis=1))

rotogrinder_pts_df = rotogrinder_pts_df.pivot_table(index=['date', 'player'], values='pts', columns='service')
if VERBOSE:
    display(rotogrinder_pts_df)

In [122]:
fantasy_pts_df = fantasy_df.copy()[~np.isnan(fantasy_df['player_id'])]
fantasy_pts_df['player'] = fantasy_pts_df.apply(lambda row: row.first_name + ' ' + row.last_name, axis=1)
fantasy_cols = ['date', 'team', 'pos', 'player'] + \
    [col for col in fantasy_pts_df.columns if ':' in col and 'goal' not in col]
fantasy_pts_df = fantasy_pts_df[fantasy_cols]
fantasy_pts_df = fantasy_pts_df.rename(columns=lambda col: col.split('_')[0] if '_' in col else col)
fantasy_pts_df['date'] = pd.to_datetime(fantasy_pts_df.date)

if VERBOSE:
    display(fantasy_pts_df)

In [123]:
merged_df = fantasy_pts_df.join(rotowire_pts_df, on=['date', 'player'], how='left') \
                          .join(rotogrinder_pts_df, on=['date', 'player'], how='left') \

if VERBOSE:
    display(merged_df)